In [1]:
import pandas as pd

file_path = 'point_cloud_1.txt'

try:
    # pd.read_csv cũng rất mạnh mẽ, header=None vì file không có dòng tiêu đề.
    # names=['x', 'y', 'z'] để đặt tên cho các cột.
    # sep=',' là ký tự phân tách.
    df = pd.read_csv(file_path, header=None, names=['x', 'y', 'z'], sep=',')

    print("Đọc file thành công vào DataFrame!")
    print(f"Tổng số điểm: {len(df)}")
    print("\n5 điểm đầu tiên:")
    print(df.head())

    # Để thực hiện tính toán, bạn nên chuyển DataFrame thành mảng NumPy
    point_cloud = df.values
    
    print("\nChuyển đổi thành mảng NumPy:")
    print(point_cloud[:5])


except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file tại '{file_path}'")
except Exception as e:
    print(f"Đã xảy ra lỗi khi đọc file: {e}")

Đọc file thành công vào DataFrame!
Tổng số điểm: 24922949

5 điểm đầu tiên:
         x        y       z
0 -40.4630  22.8951 -2.1514
1 -40.3050  23.0678 -2.1724
2 -40.3183  23.0453 -2.1444
3 -40.3126  23.0710 -2.1444
4 -40.2875  23.0910 -2.1434

Chuyển đổi thành mảng NumPy:
[[-40.463   22.8951  -2.1514]
 [-40.305   23.0678  -2.1724]
 [-40.3183  23.0453  -2.1444]
 [-40.3126  23.071   -2.1444]
 [-40.2875  23.091   -2.1434]]


In [2]:
df_sorted = df.sort_values(by='z')

# reset_index(drop=True) là một bước tùy chọn nhưng rất nên làm.
# Nó tạo lại index từ 0, 1, 2,... cho DataFrame mới, tránh nhầm lẫn.
df_sorted = df_sorted.reset_index(drop=True)

print("\n--- DataFrame sau khi đã sắp xếp theo Z ---")
print(df_sorted)



--- DataFrame sau khi đã sắp xếp theo Z ---
                x        y        z
0         -0.4364   0.0204  -2.2314
1        -40.3050  23.0678  -2.1724
2        -40.4630  22.8951  -2.1514
3        -40.3126  23.0710  -2.1444
4        -40.3183  23.0453  -2.1444
...           ...      ...      ...
24922944   5.0236   9.5987  15.4336
24922945   4.7522   9.9014  15.4356
24922946   4.7365   9.9120  15.4376
24922947   5.0630   9.5364  15.4446
24922948   4.7421   9.9078  15.4456

[24922949 rows x 3 columns]


In [3]:
df_sorted.min

<bound method DataFrame.min of                 x        y        z
0         -0.4364   0.0204  -2.2314
1        -40.3050  23.0678  -2.1724
2        -40.4630  22.8951  -2.1514
3        -40.3126  23.0710  -2.1444
4        -40.3183  23.0453  -2.1444
...           ...      ...      ...
24922944   5.0236   9.5987  15.4336
24922945   4.7522   9.9014  15.4356
24922946   4.7365   9.9120  15.4376
24922947   5.0630   9.5364  15.4446
24922948   4.7421   9.9078  15.4456

[24922949 rows x 3 columns]>

In [ ]:
import pandas as pd
import numpy as np


# 1. Xác định các tham số
target_z = 7.26    # Cao độ Z bạn muốn lấy
slice_thickness = 5 # Độ dày lớp cắt (ví dụ: 1mm). Bạn có thể điều chỉnh.

# 2. Tính toán biên trên và biên dưới của lớp cắt
lower_bound = target_z - (slice_thickness / 2)
upper_bound = target_z + (slice_thickness / 2)

print(f"\nĐang lọc các điểm có Z trong khoảng [{lower_bound:.5f}, {upper_bound:.5f}]")

# 3. Thực hiện lọc bằng boolean indexing
# Đây là cách làm hiệu quả nhất trong Pandas
slice_df = df_sorted[(df_sorted['z'] >= lower_bound) & (df_sorted['z'] <= upper_bound)]

print("\n--- Các điểm thỏa mãn điều kiện ---")
print(slice_df)

# 4. Lấy riêng tọa độ X, Y nếu cần
if not slice_df.empty:
    # Lấy X, Y dưới dạng DataFrame
    xy_coordinates_df = slice_df[['x', 'y']]
    print("\n--- Tọa độ X, Y (dạng DataFrame) ---")
    print(xy_coordinates_df)
    
    # Lấy X, Y dưới dạng mảng NumPy (thường dùng cho tính toán)
    xy_coordinates_np = slice_df[['x', 'y']].values
    print("\n--- Tọa độ X, Y (dạng mảng NumPy) ---")
    print(xy_coordinates_np)
else:
    print("\nKhông tìm thấy điểm nào trong khoảng Z đã cho.")

In [ ]:
import numpy as np

def circle_fit_by_taubin(points):
    """
    Khớp hình tròn bằng thuật toán Taubin, chuyển đổi từ code C++ của Nikolai Chernov.

    Args:
        points (np.array): Một mảng NumPy có hình dạng (N, 2), 
                           trong đó N là số điểm, các cột là X và Y.

    Returns:
        tuple: (a, b, r, iterations) 
               a, b: Tọa độ tâm hình tròn
               r: Bán kính hình tròn
               iterations: Số lần lặp của phương pháp Newton
    """
    n = points.shape[0]
    if n < 3:
        raise ValueError("Cần ít nhất 3 điểm để khớp hình tròn.")

    # --- 1. Tính toán giá trị trung bình và dịch chuyển dữ liệu ---
    mean_x = np.mean(points[:, 0])
    mean_y = np.mean(points[:, 1])
    
    # Dữ liệu đã được dịch chuyển (centered data)
    xi = points[:, 0] - mean_x
    yi = points[:, 1] - mean_y
    zi = xi**2 + yi**2

    # --- 2. Tính toán các Moment ---
    # np.mean() sẽ thực hiện cả việc cộng và chia cho n
    m_xx = np.mean(xi**2)
    m_yy = np.mean(yi**2)
    m_xy = np.mean(xi * yi)
    m_xz = np.mean(xi * zi)
    m_yz = np.mean(yi * zi)
    m_zz = np.mean(zi**2)

    # --- 3. Tính các hệ số của Phương Trình Đặc Trưng ---
    m_z = m_xx + m_yy
    cov_xy = m_xx * m_yy - m_xy**2
    var_z = m_zz - m_z**2
    
    a3 = 4 * m_z
    a2 = -3 * m_z**2 - m_zz
    a1 = var_z * m_z + 4 * cov_xy * m_z - m_xz**2 - m_yz**2
    a0 = m_xz * (m_xz * m_yy - m_yz * m_xy) + m_yz * (m_yz * m_xx - m_xz * m_xy) - var_z * cov_xy
    
    a22 = a2 + a2
    a33 = a3 + a3 + a3
    
    # --- 4. Tìm Nghiệm của Đa Thức (Newton's Method) ---
    ITER_MAX = 1000
    x = 0.0  # Bắt đầu từ x=0
    y = a0
    
    for i in range(ITER_MAX):
        # Tính đạo hàm P'(x)
        dy = a1 + x * (a22 + a33 * x)
        if dy == 0: # Tránh chia cho 0
            break

        x_new = x - y / dy
        
        # Kiểm tra điều kiện dừng
        if x_new == x or not np.isfinite(x_new):
            break
            
        # Tính giá trị mới của đa thức P(x_new)
        y_new = a0 + x_new * (a1 + x_new * (a2 + x_new * a3))
        
        if abs(y_new) >= abs(y):
            break
            
        x = x_new
        y = y_new
        
    iterations = i + 1

    # --- 5. Tính toán các tham số của hình tròn ---
    det = x**2 - x * m_z + cov_xy
    if det == 0:
        return None, None, None, iterations # Không thể giải
    
    # Tâm trong hệ tọa độ đã dịch chuyển
    x_center = (m_xz * (m_yy - x) - m_yz * m_xy) / (2 * det)
    y_center = (m_yz * (m_xx - x) - m_xz * m_xy) / (2 * det)

    # --- 6. Tổng hợp kết quả ---
    # Dịch chuyển tâm ngược lại về hệ tọa độ gốc
    a = x_center + mean_x
    b = y_center + mean_y
    r = np.sqrt(x_center**2 + y_center**2 + m_z)

    return a, b, r, iterations

# --- Ví dụ sử dụng ---
if __name__ == '__main__':

    x_points = slice_df['x']
    y_points = slice_df['y']
    
    data_points = np.c_[x_points, y_points]

    # Khớp hình tròn bằng thuật toán Taubin
    a_fit, b_fit, r_fit, iters = circle_fit_by_taubin(data_points)

    print("--- Kết quả khớp hình tròn bằng thuật toán Taubin ---")
    print(f"Giá trị khớp: a={a_fit:.4f}, b={b_fit:.4f}, r={r_fit:.4f}")
    print(f"Số lần lặp của Newton's method: {iters}")



In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, figsize=(8, 8))

fitting_circle = plt.Circle(
    (a_fit, b_fit),      # Tọa độ tâm
    r_fit,               # Bán kính
    color='red',         # Màu của đường viền
    fill=False,          # Không tô màu bên trong
    linewidth=2,         # Độ dày của đường viền
    label='Fitting Circle' # Nhãn cho chú thích
)

# 3. Thêm (vẽ) đối tượng hình tròn lên Axes
ax.add_artist(fitting_circle)

# 4. (Tùy chọn) Vẽ các điểm dữ liệu gốc để so sánh
if 'x_points' in locals():
    ax.scatter(x_points, y_points, label='Data Points')

# 5. (Tùy chọn) Vẽ tâm của hình tròn để dễ nhìn
ax.plot(a_fit, b_fit, 'r+', markersize=10, label='Center')

# 6. Thiết lập các thuộc tính cho đồ thị
ax.set_title('Vẽ đường tròn đã biết')
ax.set_xlabel('Tọa độ X')
ax.set_ylabel('Tọa độ Y')
ax.legend()  # Hiển thị chú thích
ax.grid(True) # Bật lưới

# 7. Đặt tỉ lệ các trục bằng nhau - RẤT QUAN TRỌNG!
# Điều này đảm bảo hình tròn trông giống hình tròn, không phải hình elip.
ax.set_aspect('equal', adjustable='box')

# 8. Tự động điều chỉnh giới hạn của các trục để hình tròn nằm gọn trong khung nhìn
# Thêm một chút lề (padding) để đẹp hơn
padding = r_fit * 0.1
ax.set_xlim(a_fit - r_fit - padding, a_fit + r_fit + padding)
ax.set_ylim(b_fit - r_fit - padding, b_fit + r_fit + padding)

# 9. Hiển thị đồ thị
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

# (Dán các hàm circle_fit_by_taubin và circle_residuals vào đây...)
# ...
def circle_fit_by_taubin(points):
    """Khớp hình tròn bằng thuật toán Taubin."""
    n = points.shape[0]
    if n < 3: return None, None, None, 0
    mean_x = np.mean(points[:, 0])
    mean_y = np.mean(points[:, 1])
    xi = points[:, 0] - mean_x
    yi = points[:, 1] - mean_y
    zi = xi**2 + yi**2
    m_xx = np.mean(xi**2); m_yy = np.mean(yi**2); m_xy = np.mean(xi * yi)
    m_xz = np.mean(xi * zi); m_yz = np.mean(yi * zi); m_zz = np.mean(zi**2)
    m_z = m_xx + m_yy; cov_xy = m_xx * m_yy - m_xy**2; var_z = m_zz - m_z**2
    a3 = 4 * m_z; a2 = -3 * m_z**2 - m_zz
    a1 = var_z * m_z + 4 * cov_xy * m_z - m_xz**2 - m_yz**2
    a0 = m_xz * (m_xz * m_yy - m_yz * m_xy) + m_yz * (m_yz * m_xx - m_xz * m_xy) - var_z * cov_xy
    a22 = a2 + a2; a33 = a3 + a3 + a3
    x = 0.0; y = a0; i = 0
    for i in range(99):
        dy = a1 + x * (a22 + a33 * x)
        if dy == 0: break
        x_new = x - y / dy
        if x_new == x or not np.isfinite(x_new): break
        y_new = a0 + x_new * (a1 + x_new * (a2 + x_new * a3))
        if abs(y_new) >= abs(y): break
        x = x_new; y = y_new
    iterations = i + 1
    det = x**2 - x * m_z + cov_xy
    if det == 0: return None, None, None, iterations
    x_center = (m_xz * (m_yy - x) - m_yz * m_xy) / (2 * det)
    y_center = (m_yz * (m_xx - x) - m_xz * m_xy) / (2 * det)
    a = x_center + mean_x; b = y_center + mean_y
    r = np.sqrt(x_center**2 + y_center**2 + m_z)
    return a, b, r, iterations

def circle_residuals(params, x_data, y_data):
    """Hàm tính vector phần dư hình học."""
    a, b, R = params
    return np.sqrt((x_data - a)**2 + (y_data - b)**2) - R

def circle_fit_by_lm(points, initial_guess):
    """Khớp hình tròn bằng Levenberg-Marquardt."""
    x_data = points[:, 0]
    y_data = points[:, 1]
    res = least_squares(circle_residuals, initial_guess, args=(x_data, y_data), method='lm')
    a_fit, b_fit, r_fit = res.x
    return a_fit, b_fit, r_fit

# ==============================================================================
# HÀM MỚI: TÍNH TOÁN CÁC CHỈ SỐ SAI SỐ
# ==============================================================================
def calculate_errors(points, circle_params):
    """
    Tính toán các chỉ số sai số cho kết quả khớp hình tròn.

    Args:
        points (np.array): Mảng điểm dữ liệu (N, 2).
        circle_params (tuple): Tham số của hình tròn đã khớp (a, b, R).

    Returns:
        dict: Một từ điển chứa các chỉ số sai số.
    """
    a, b, R = circle_params
    x_data = points[:, 0]
    y_data = points[:, 1]
    
    # 1. Tính toán vector phần dư (residuals)
    residuals = circle_residuals(circle_params, x_data, y_data)
    
    # 2. Tính MSE
    mse = np.mean(residuals**2)
    
    # 3. Tính RMSE (sigma)
    rmse = np.sqrt(mse)
    
    # 4. Tính MAE
    mae = np.mean(np.abs(residuals))
    
    # 5. (Tùy chọn) Tìm sai số lớn nhất để biết mức độ ảnh hưởng của outlier
    max_error = np.max(np.abs(residuals))
    
    error_metrics = {
        'residuals': residuals,
        'MSE': mse,
        'RMSE (sigma)': rmse,
        'MAE': mae,
        'Max Error': max_error,
        'Num Points': len(points)
    }
    
    return error_metrics


# ==============================================================================
# CẬP NHẬT QUY TRÌNH CHÍNH
# ==============================================================================
if __name__ == '__main__':
    # --- 1. Tạo dữ liệu mẫu ---

    data_points = np.c_[x_points, y_points]

    # --- 2. Chạy Taubin để có điểm khởi đầu ---
    print("--- Bước 1: Tìm điểm khởi đầu bằng Taubin Fit ---")
    a_taubin, b_taubin, r_taubin, _ = circle_fit_by_taubin(data_points)
    
    if a_taubin is not None:
        initial_guess = (a_taubin, b_taubin, r_taubin)
        print(f"Kết quả Taubin (Initial Guess): a={a_taubin:.4f}, b={b_taubin:.4f}, r={r_taubin:.4f}")

        # --- 3. Chạy Levenberg-Marquardt ---
        print("\n--- Bước 2: Tinh chỉnh bằng Levenberg-Marquardt Fit ---")
        a_lm, b_lm, r_lm = circle_fit_by_lm(data_points, initial_guess)
        final_params = (a_lm, b_lm, r_lm)
        print(f"Kết quả cuối cùng (LM): a={a_lm:.4f}, b={b_lm:.4f}, r={r_lm:.4f}")

        # --- 4. TÍNH TOÁN VÀ IN RA SAI SỐ ---
        print("\n--- Bước 3: Đánh giá sai số của kết quả cuối cùng ---")
        errors = calculate_errors(data_points, final_params)
        
        # In các chỉ số chính
        print(f"Số điểm dữ liệu: {errors['Num Points']}")
        print(f"Sai số bình phương trung bình gốc (RMSE): {errors['RMSE (sigma)']:.6f}")
        print(f"Sai số tuyệt đối trung bình (MAE): {errors['MAE']:.6f}")
        print(f"Sai số lớn nhất (gây ra bởi outlier): {errors['Max Error']:.4f}")
        
        # (Tùy chọn) In ra 5 phần dư đầu tiên
        # print("\n5 phần dư đầu tiên:")
        # print(errors['residuals'][:5])

        # --- 5. Vẽ đồ thị ---
        fig, ax = plt.subplots(1, figsize=(10, 10))
        ax.scatter(data_points[:, 0], data_points[:, 1], label='Data Points')
        circle_taubin = plt.Circle((a_taubin, b_taubin), r_taubin, color='orange', fill=False, linestyle='--', linewidth=2, label='Taubin Fit (Initial Guess)')
        ax.add_artist(circle_taubin)
        circle_lm = plt.Circle(final_params[:2], final_params[2], color='red', fill=False, linewidth=2, label='Levenberg-Marquardt Fit (Final)')
        ax.add_artist(circle_lm)
        ax.set_title('So sánh Taubin Fit và Levenberg-Marquardt Fit')
        ax.set_xlabel('Tọa độ X'); ax.set_ylabel('Tọa độ Y')
        ax.legend(); ax.grid(True); ax.set_aspect('equal', adjustable='box')
        plt.show()